### Tuning model: es basicamente reentrenar el clasificador de la etapa 1. Para conocer cual es el mejor clasificador que se puede optener con el espacio de caracteristicas optenido.



Notebook para ver como congelar los parametros que no queremos modificar.

In [2]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self):           
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( # input shape (3, 32, 32)
        nn.Conv2d(
        in_channels=3, # input height
        out_channels=16, # n_filters
        kernel_size=5, # filter size
        stride=1, # filter movement/step
        padding=2,
        # if want same width and length of this image after con2d,
        #padding=(kernel_size-1)/2 if stride=1
        ), # output shape (16, 16, 16)
        nn.ReLU(), # activation
        nn.MaxPool2d(kernel_size=2),
        # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential( # input shape (16, 14, 14)
        nn.Conv2d(16, 32, 5, 1, 2), # output shape (32, 8, 8)
        nn.ReLU(), # activation
        nn.MaxPool2d(2), # output shape (32, 7, 7)
        )
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32 *8 *8, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        # flatten the output of conv2 to (batch_size, 32 * 8 * 8)
        output = self.linear_relu_stack(x)
        return output # return x for visualizatio

# Crear instancia del modelo
model = CNN()

# Cargar el estado previamente guardado
fase_1 = torch.load('Fase_1.pth')

# Cargar los pesos al modelo
model.load_state_dict(fase_1)

<All keys matched successfully>

Queremos congelar todas las capas menos las ultimas dos capas que son el bias y los pesos.

In [5]:
print(len(fase_1))
for x,i in enumerate(model.parameters()):
    print(x,i.shape)

10
0 torch.Size([16, 3, 5, 5])
1 torch.Size([16])
2 torch.Size([32, 16, 5, 5])
3 torch.Size([32])
4 torch.Size([512, 2048])
5 torch.Size([512])
6 torch.Size([512, 512])
7 torch.Size([512])
8 torch.Size([10, 512])
9 torch.Size([10])


In [6]:
for idx,layer in enumerate(model.parameters()):
    if idx == len(fase_1) -2: break
    layer.requires_grad = False

In [7]:
# Verificar qué capas están congeladas
for idx, capa in enumerate(model.parameters()):
    print(idx,capa.requires_grad)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 True
9 True
